In [1]:
import numpy
import datetime
import pandas
from datetime import date
import math
import numpy as np
from sympy.solvers import solve
from sympy import Symbol
import random
import tqdm as tqdm

In [2]:
class MarketData:
    def __init__(self, date: datetime.datetime):
    # инициализация экземпляра класса, рыночные данные берутся за определенную дату
        converted = '{}-{}-{}'.format(date.day, date.strftime("%b"), date.year)
        self.data = {}
        temp_data = pandas.read_csv('data.csv', header=None)
        temp_data_dict = temp_data.to_dict()
        for key in temp_data_dict[0].keys():
            self.data[temp_data_dict[0][key]] = temp_data_dict[1][key].strip()
        
        if converted != self.data['Date'].strip():
            print('No such date in data! Error!')
        
        self.cds_curves = {}
        self.recovery = None
        self.cds_currencies = {}
        self.zero_rates = {}
        for key, value in self.data.items():
            if key[:3] == 'CDS':
                spread = key.split('_')[3] == 'SPREAD'
                if spread:
                    company_name = key.split('_')[1]
                    currency = key.split('_')[2]
                    period = key.split('_')[4]

                    if company_name not in self.cds_currencies:
                        self.cds_currencies[company_name] = currency
                        
                    if company_name not in self.cds_curves:
                        self.cds_curves[company_name] = {}
#                     if currency not in self.cds_curve[company_name]:
#                         self.cds_curve[company_name][currency] = {}
#                     self.cds_curve[company_name][currency][convert_period_to_years(period)] = float(value)
                    self.cds_curves[company_name][self.convert_period_to_years(period)] = float(value)
                if key.split('_')[3] == 'RECOVERY':
                    self.recovery = float(value)
            if key[:2] == 'IR':
#                 'IR_USD_ZERORATE_6M': '0.0006',
                currency = key.split('_')[1]
                period = key.split('_')[3]
                if currency not in self.zero_rates:
                    self.zero_rates[currency] = {}
                self.zero_rates[currency][self.convert_period_to_years(period)] = float(value)
    
    def get_cds_curve(self, name: str):
    # метод возвращает кривую спредов CDS для заданного имени в формате словаря {term: spread), где term -- срок в годах, spread -- спред в бипсах
        if name in self.cds_curves:
            return self.cds_curves[name]
        else:
            print('No such company in cds')
            return {}
    
    def get_cds_recovery(self, name: str):
    # метод возвращает численный recovery rate для заданного имени
        return self.recovery
    
    def get_cds_currency(self, name: str):
    # метод возвращает валюту CDS для заданного имени
        if name in self.cds_currencies:
            return self.cds_currencies[name]
        else:
            print('No such company in cds')
            return ''

    def get_zero_rates(self, currency: str):
    # метод возвращает кривую zero rates для заданной валюты в формате словаря {term: rate), где term -- срок в годах, rate -- численная ставка
        if currency in self.zero_rates:
            return self.zero_rates[currency]
        else:
            print('No such currency in zero rates')
            return {}
            
    @staticmethod
    def convert_period_to_years(period: str):
        # Конвертирует строки формата 7M и 3Y в количество лет
        if period[-1] == 'M':
            return(float(period[:-1])/12)
        elif period[-1] == 'Y':
            return(float(period[:-1]))
        else:
            print('Smth strange! Error')
            return 0

class InterestRates:
    @staticmethod
    def get_interpolated_rate(md: MarketData, currency: str, times: list or numpy.array):
    # метод возвращает линейно интерполированную ставку для заданной валюты и соответствующих сроков в формате numpy.array (рекомендуется использовать функцию numpy.interp)
        if currency in md.zero_rates:
            x = sorted(md.zero_rates[currency].keys())
            y = [md.zero_rates[currency][period] for period in x]
            return numpy.interp(times, x, y)
        else:
            print('No such currency in zero rates')
            return numpy.array()

    @staticmethod
    def get_discounting(md: MarketData, currency: str, times: list or numpy.array, compounding: int):
    # метод возвращает кривую дисконт факторов для заданной валюты, соответствующих сроков и частоты капитализации (compounding) в формате numpy.array
    # compounding=1 => капитализация ежегодная, compounding=n => капитализация n раз в год, но (!) compounding=0 => капитализация непрерывная
        pass

    

In [3]:
   def get_cds_recovery(self, name: str):
    # метод возвращает численный recovery rate для заданного имени
        return self.recovery

In [4]:
if  get_cds_recovery != 0:
    print("False") 
else:
    print('Ok')

False


In [5]:
md = MarketData(datetime.datetime(2021,10,28))

In [92]:
class Credit:
    
    
    @staticmethod
    def get_bootstrapped_sp(md: MarketData, name: str):
        #возвращает кривую спредов CDS для заданного имени в формате словаря {term: spread), где term -- срок в годах, spread -- спред в бипсах
        recovery_rate = md.get_cds_recovery(name)
        cds_spreads = md.get_cds_curve(name)
        surv_probs = {}
        for term in cds_spreads:
            surv_prob = np.exp(((-cds_spreads[term] / 10000)*term) / (1 - recovery_rate))
            surv_probs[term] = surv_prob
        return surv_probs
    
    @staticmethod
    def (md: MarketData, name: str, times: list):
    # возвращающий для эмитента лог-линейно интерполированную survival probability по заданному набору сроков (в годах) в формате numpy.array.
        fp = np.log(list(Credit.get_bootstrapped_sp(md, name).values()))
        xp = list(Credit.get_bootstrapped_sp(md, name).keys())
        x = times
        return np.e ** np.interp(x, xp, fp)
    
    # линейное интерполирвание спредов
    @staticmethod
    def get_interpolated_spread(md: MarketData, name: str, times: list):
        fp = list(md.get_cds_curve('JAGUAR').values())
        xp = list(md.get_cds_curve('JAGUAR').keys())
        x = times
        return np.interp(x, xp, fp)
    
    
    
    @staticmethod
    def price_cds(md: MarketData, trade_date: datetime.datetime, name: str, spread: float, term: float, frequency: int, currency: str, side: str):
        R = md.recovery
        Protection_Leg = (1 - md.recovery) / 2
        Premium_Leg = spread / 10000
        RPVO1 = 0
        dates_of_payments = []
        day_of_previous_payment = datetime.datetime(trade_date.year, 1, 20)

        # рассматриваем 4 стандартных даты выплаты
        # определим дату предыдущего платежа в момент заключения сделки
        Count_Of_Payments = 0
        if trade_date.month % 3 == 0 and trade_date.day == 20:
            Count_Of_Payments = math.ceil(term * frequency)
            day_of_previous_payment = trade_date
        else:
            Count_Of_Payments = math.ceil(term * frequency) + 1
            if trade_date.month > 3 and trade_date.month < 6:
                day_of_previous_payment = datetime.datetime(trade_date.year, 3, 20)
            elif trade_date.month > 6 and trade_date.month < 9:
                day_pf_previous_payment = datetime.datetime(trade_date.year, 6, 20)
            elif trade_date.month > 9 and trade_date.month < 12:
                day_of_previous_payment = datetime.datetime(trade_date.year, 9, 20)
            else:
                day_of_previous_payment = datetime.datetime(trade_date.year - 1, 12, 20)
                
        # определим даты платежей
        dates_of_payments = [day_of_previous_payment]
        for steps in range(1, Count_Of_Payments + 1):
            month = dates_of_payments[-1].month
            year = dates_of_payments[-1].year
            if month + 6 > 12:
                month = (month + 6)%12
                year += 1
            else:
                month += 6
            dates_of_payments.append(datetime.datetime(year, month, dates_of_payments[-1].day))
        
        # посчитаем количество дней между соответствующими датами платежей
        dates_diff = [(trade_date - day_of_previous_payment).days]
        dates_diff.append((dates_of_payments[1] - trade_date).days)
        for i in range(1, len(dates_of_payments) - 1):
            dates_diff.append((dates_of_payments[i + 1] - dates_of_payments[i]).days)
        
        # выпишем ставки платежей
        rates_of_payments = []
        cum_dates = np.cumsum(dates_diff)
        rates_of_payments = InterestRates.get_interpolated_rate(md, currency, cum_dates/365.25)
        
        # выпишем соответсвующие вероятности
        probabilities = []
        probabilities = Credit.get_interpolated_sp(md, name, cum_dates/365.25)
        
        # Рассчитаем дисконты
        discount_factors = []
        time = cum_dates/365.25
        for i in range(len(rates_of_payments)):
            discount_factors.append(1 / (1 + rates_of_payments[i]) ** time[i])

        
        # вычислим RPVO1
        dates_diff = np.array(dates_diff, dtype='float')
        dates_diff /= 365.25
        RPVO1 = dates_diff[0] * discount_factors[0] * (1 - probabilities[0])
        RPVO1 += 0.5 * dates_diff[1] * discount_factors[0] * (1 - probabilities[0])
        RPVO1 += (dates_diff[0] + dates_diff[1]) * discount_factors[0] * probabilities[0]
        for i in range(1, len(discount_factors) - 1):
            RPVO1 += 0.5 * dates_diff[i + 1] * discount_factors[i] * (probabilities[i - 1] + probabilities[i])
        
        # premium leg
        Premium_Leg *= RPVO1
        
        # protection leg
        # для допустимого уровня погрешности нужны ежемесячные временные шаги
        # для этого получим новые интерполированные ставки по месяцам
        new_rates = []
        new_dates = []
        for i in range(1, math.ceil(12 * term)):
            new_dates.append(i/12)
        new_rates = InterestRates.get_interpolated_rate(md, currency, new_dates)
        
        # дисконты
        discount_factors_new = []
        for i in range(len(new_rates)):
            discount_factors_new.append(1 / (1 + new_rates[i]) ** new_dates[i])
        
        # получим новые вероятности
        new_probabilities = Credit.get_interpolated_sp(md, name, new_dates)
        
        # вычислим Protection Leg
        res = 0
        for i in range(len(discount_factors_new) - 1):
            res += (discount_factors_new[i] + discount_factors_new[i + 1]) * (new_probabilities[i] - new_probabilities[i + 1])
        Protection_Leg *= res
        
        V = Protection_Leg - Premium_Leg
        
        if side == 'buy':
            return V
        elif side == 'sell':
            return -V

    @staticmethod 
    def price_cln(md: MarketData, trade_date: datetime.datetime, name: str, coupon: float, term: float, frequency: int, currency: str, pay_accrued: bool, recovery: float):
        nominal = 1
        recovery_rate = md.recovery
        dates = []
        day_of_previous_payment = trade_date
    
        N = 0
        if trade_date.month % 3 == 0 and trade_date.day == 20:
            N = math.ceil(term * frequency)
        else:
            N = math.ceil(term * frequency) + 1
            if trade_date.month > 3 and trade_date.month < 6:
                day_of_previous_payment = datetime.datetime(trade_date.year, 3, 20)
            elif trade_date.month > 6 and trade_date.month < 9:
                day_of_previous_payment = datetime.datetime(trade_date.year, 6, 20)
            elif trade_date.month > 9 and trade_date.month < 12:
                day_of_previous_payment = datetime.datetime(trade_date.year, 9, 20)
            else:
                day_of_previous_payment = datetime.datetime(trade_date.year - 1, 12, 20)    
            
        dates = [day_of_previous_payment]
        for i in range(N):
            month = dates[-1].month
            year = dates[-1].year
            if month + 6 > 12:
                month = (month + 6)%12
                year += 1
            else:
                month += 6
            dates.append(datetime.datetime(year, month, dates[-1].day))
            
        dates_diff = [(trade_date - day_of_previous_payment).days]
        dates_diff.append((dates[1] - trade_date).days)
    
        for i in range(1, len(dates) - 1):
            dates_diff.append((dates[i + 1] - dates[i]).days)
        
        cum_dates = np.cumsum(dates_diff)
    
        rates = []
        rates = InterestRates.get_interpolated_rate(md, currency, cum_dates/365.25)
        
        probabilities = []
        probabilities = Credit.get_interpolated_sp(md, name, cum_dates/365.25)
        
        dates = cum_dates / 365.25
        discount_factors1 = []
        for i in range(len(rates)):
            discount_factors1.append(1 / (1 + rates[i]) ** dates[i])
    
        value = rates[-1]*probabilities[-1]
        for t in range(len(discount_factors1) - 1):
            value += nominal*(discount_factors1[t+1]*probabilities[t+1]*coupon / 100) + discount_factors1[t+1]*(probabilities[t] - probabilities[t+1])*recovery_rate
        
        return value
    
    # Европейский CDS: protection leg платится в момент экспирации контракта
    # (более упрощенный прайсер по сравнению с первым)
    @staticmethod
    def price_european_cds(md: MarketData, name: str, term: float, frequency: int, currency: str, side: str, coupon: float):
        
        # периоды платежей
        periods_of_premium_payments = []
        i = 0
        while i <= term:
            i += 1 / frequency
            periods_of_premium_payments.append(i)
        
        # посчитаем соответствующие вероятности
        survival_prob = []
        survival_prob = Credit.get_interpolated_sp(md, name, periods_of_premium_payments)
        
        # посчитаем коэффициенты дисконтирования для платежей
        rates = InterestRates.get_interpolated_rate(md, currency, periods_of_premium_payments)
        discount_factors = []
        for i in range(len(rates)):
            discount_factors.append(1 / (1 + rates[i]) ** periods_of_premium_payments[i])
        
        # premium leg
        premium_leg = 0 
        for i in range(len(rates)):
            premium_leg += ((coupon / 100) / frequency) * discount_factors[i] * survival_prob[i]
            
        # protection leg
        R = md.recovery
        protection_leg = (1 - R) * (1 - survival_prob[-1]) * discount_factors[-1]
        
        # цена CDS
        V = protection_leg - premium_leg
        if side == 'buy':
            return V
        elif side == 'sell':
            return -V
        
        
    # Американский CDS
    # Protection leg платится после кредитного события
    # в этом случае premium leg рассчитывается так же, как и в европейском CDS
    # а protection leg считается как в первоначальном прайсере, учитывая вероятности дефолта на каждом интервале и приближение
    @staticmethod
    def price_american_cds(md: MarketData, name: str, term: float, frequency: int, currency: str, side: str, coupon: float):
        
        # периоды платежей
        periods_of_premium_payments = []
        i = 0
        while i <= term:
            i += 1 / frequency
            periods_of_premium_payments.append(i)
        
        # посчитаем соответствующие вероятности
        survival_prob = []
        survival_prob = Credit.get_interpolated_sp(md, name, periods_of_premium_payments)
        
        # посчитаем коэффициенты дисконтирования для платежей
        rates = InterestRates.get_interpolated_rate(md, currency, periods_of_premium_payments)
        discount_factors = []
        for i in range(len(rates)):
            discount_factors.append(1 / (1 + rates[i]) ** periods_of_premium_payments[i])
        
        # premium leg
        premium_leg = 0 
        for i in range(len(rates)):
            premium_leg += ((coupon / 100) / frequency) * discount_factors[i] * survival_prob[i]
        
        # protection leg
        protection_leg = (1 - md.recovery) / 2
        
        # для допустимого уровня погрешности нужны ежемесячные временные шаги
        # для этого получим новые интерполированные ставки по месяцам
        new_rates = []
        new_dates = []
        for i in range(1, math.ceil(12 * term)):
            new_dates.append(i/12)
        new_rates = InterestRates.get_interpolated_rate(md, currency, new_dates)
        
        discount_factors_new = []
        for i in range(len(new_rates)):
            discount_factors_new.append(1 / (1 + new_rates[i]) ** new_dates[i])

        
        # получим новые вероятности
        new_probabilities = Credit.get_interpolated_sp(md, name, new_dates)
        
        # вычислим Protection Leg
        res = 0
        for i in range(len(discount_factors_new) - 1):
            res += (discount_factors_new[i] + discount_factors_new[i + 1]) * (new_probabilities[i] - new_probabilities[i + 1])
        protection_leg *= res
        
        V = protection_leg - premium_leg
        
        if side == 'buy':
            return V
        elif side == 'sell':
            return -V
    
    # реализация прайсера CMCDS 
    # не смог найти данные по спредам cds корпоративных облигаций, поэтому буду считать по старым данным с ягуаром
    # индексируемый спред будем считать на основе американского CDS    
        
        
    @staticmethod
    def price_cmcds(md: MarketData, name: str, term: float, frequency: int, currency: str, side: str, mode: bool):
        
        Protection_Leg_1 = 0
        Premium_Leg_1 = 0
        Premium_Leg = 0
        Protection_Leg = 0
        
        # Protection Leg не имеет изменений по сравнению с прошлыми рассчетами, так что выпишем сначала его
        
        # protection leg
        Protection_Leg = (1 - md.recovery) / 2
        
        # для допустимого уровня погрешности нужны ежемесячные временные шаги
        # для этого получим новые интерполированные ставки по месяцам
        new_rates = []
        new_dates = []
        for i in range(1, math.ceil(12 * term)):
            new_dates.append(i/12)
        new_rates = InterestRates.get_interpolated_rate(md, currency, new_dates)
        
        discount_factors_new = []
        for i in range(len(new_rates)):
            discount_factors_new.append(1 / (1 + new_rates[i]) ** new_dates[i])

        
        # получим новые вероятности
        new_probabilities = Credit.get_interpolated_sp(md, name, new_dates)
        
        # вычислим Protection Leg
        res = 0
        for i in range(len(discount_factors_new) - 1):
            res += (discount_factors_new[i] + discount_factors_new[i + 1]) * (new_probabilities[i] - new_probabilities[i + 1])
        Protection_Leg *= res
        
        # теперь рассчитаем плавающие спреды
        
        # периоды платежей
        periods_of_premium_payments = []
        i = 0
        while i <= term:
            i += 1 / frequency
            periods_of_premium_payments.append(i)
        
        # посчитаем соответствующие вероятности
        survival_prob = []
        survival_prob = Credit.get_interpolated_sp(md, name, periods_of_premium_payments)
        
        # Рассчитаем форвардные ставки с даты предыдущего платежа на дату следующего платежа
        
        rates = InterestRates.get_interpolated_rate(md, currency, periods_of_premium_payments)
        
        fwd_rates = []
        fwd_rates.append(rates[0])
        
        for i in range(1, len(periods_of_premium_payments) - 1):
            fwd_rates.append((((1 + rates[i + 1]) ** periods_of_premium_payments[i + 1]) / ((1 + rates[i]) ** periods_of_premium_payments[i])) ** (1 / (periods_of_premium_payments[i + 1] - periods_of_premium_payments[i])) - 1)
        
        # выпишем соответствующие дискаунт факторы
        
        discount_factors = [1 / (1 + fwd_rates[0]) ** periods_of_premium_payments[0]]
        for i in range(1, len(periods_of_premium_payments) - 1):
            discount_factors.append(1 / (1 + fwd_rates[i]) ** (periods_of_premium_payments[i] - periods_of_premium_payments[i - 1]))
        
        floating_spread = []
        
        if mode == 0:
        
            x = Symbol('x')


            for t in range(len(periods_of_premium_payments) - 1):
                Premium_Leg_1 = (x / frequency) * discount_factors[t] * survival_prob[t]
            
                Protection_Leg_1 = (1 - md.recovery) / 2
            
                new_rates_1 = []
                new_dates_1 = []
                for i in range(1, math.ceil(12 * periods_of_premium_payments[t]) + 1):
                    new_dates_1.append(i/12)
                new_rates_1 = InterestRates.get_interpolated_rate(md, currency, new_dates_1)
        
                discount_factors_new_1 = []
                for i in range(len(new_rates_1)):
                    discount_factors_new_1.append(1 / (1 + new_rates_1[i]) ** new_dates_1[i])

        
                # получим новые вероятности
                new_probabilities_1 = Credit.get_interpolated_sp(md, name, new_dates_1)
        
                # вычислим Protection Leg
                res = 0
                for i in range(len(discount_factors_new_1) - 1):
                    res += (discount_factors_new_1[i] + discount_factors_new_1[i + 1]) * (new_probabilities_1[i] - new_probabilities_1[i + 1])
                Protection_Leg_1 *= res
            
                floating_spread.append(solve(Protection_Leg_1 - Premium_Leg_1, x))
            
            # Теперь посчитаем итоговую премиальную ногу с плавающими спредами
        
            floating_spread = np.array(floating_spread)
            floating_spread /= 100
        
            for i in range(len(floating_spread)):
                Premium_Leg += floating_spread[i] * discount_factors[i] * survival_prob[i]
            Premium_Leg = float(Premium_Leg)
                
        elif mode == 1:
            
            floting_spread = []
            for i in range(len(periods_of_premium_payments)):
                floating_spread.append(random.lognormvariate(mu = 0, sigma = 0.01 * math.sqrt(periods_of_premium_payments[i])))
                
            floating_spread = np.array(floating_spread)
            floating_spread /= 100
                
            for i in range(len(floating_spread) - 1):
                Premium_Leg += floating_spread[i] * discount_factors[i] * survival_prob[i]
            Premium_Leg = float(Premium_Leg)

  
                
        V = Protection_Leg - Premium_Leg
        
        if side == 'buy':
            return V
        elif side == 'sell':
            return -V
        
        @staticmethod
    def price_cds_fwd(md: MarketData, name: str, term: float, frequency: int, currency: str, fwd_period: float, side: str):
        
        # рассчитаем форвардный спред
        # контракт сроком на term через fwd_period
        
        # даты платежей
        
        i = fwd_period
        dates_of_premium_payments = [i]
        while i < term:
            i += 1 / frequency
            dates_of_premium_payments.append(i)
        
        # спотовые ставки на эти даты
        spot_rates = InterestRates.get_interpolated_rate(md, currency, dates_of_premium_payments)

        # форвардные ставки
        
        fwd_rates = []
        for i in range(1, len(spot_rates)):
            fwd_rates.append((((1 + spot_rates[i]) ** dates_of_premium_payments[i]) / 
                            ((1 + spot_rates[0]) ** dates_of_premium_payments[0])) ** 
                            (1 / (dates_of_premium_payments[i] - dates_of_premium_payments[0])) - 1)
        
        # посчитаем дисконты на эти ставки
        
        discount_factors = []
        for i in range(len(fwd_rates)):
            discount_factors.append(1 / (1 + fwd_rates[i]) ** (dates_of_premium_payments[i + 1] - dates_of_premium_payments[0]))
        
        
        # посчитаем вероятности
        
        surv_prob = Credit.get_interpolated_sp(md, name, dates_of_premium_payments)
        
        # посчитаем premium leg
        
        Premium_Leg = 0
        
        x = Symbol('x')
        
        for t in range(len(discount_factors)):
                Premium_Leg = (x / frequency) * discount_factors[t] * surv_prob[t + 1]
                
        
        # посчитаем protection leg
        
        Protection_Leg = (1 - 0.4) / 2
        
        # для допустимого уровня погрешности нужны ежемесячные временные шаги
        # для этого получим новые интерполированные ставки по месяцам
        new_rates = []
        new_dates = []
        for i in range(math.ceil(12 * fwd_period), math.ceil(12 * term)):
            new_dates.append(i/12)
        new_rates = InterestRates.get_interpolated_rate(md, currency, new_dates)
        

         # получим новые вероятности
        new_probabilities = Credit.get_interpolated_sp(md, name, new_dates)
        
        # получим новые форвардные ставки и дисконты на них
        
        fwd_rates_new = []
        for i in range(1, len(new_rates)):
            fwd_rates_new.append((((1 + new_rates[i]) ** new_dates[i]) / 
                            ((1 + new_rates[0]) ** new_dates[0])) ** 
                            (1 / (new_dates[i] - new_dates[0])) - 1)
            
        new_df = []
        for i in range(len(fwd_rates_new)):
            new_df.append(1 / (1 + fwd_rates_new[i]) ** (new_dates[i + 1] - new_dates[0]))
        
        
        # вычислим Protection Leg
        res = 0
        for i in range(len(new_df) - 1):
            res += (new_df[i] + new_df[i + 1]) * (new_probabilities[i + 1] - new_probabilities[i + 2])
        Protection_Leg *= res
        
        # получим форвардный спред
        
        Fwd_Spread = solve(Protection_Leg - Premium_Leg, x)[0]
        
        # Теперь сделаем сам прайсер
        
        Protection_Leg_1 = 0
        Premium_Leg_1 = 0
        
        spot_discounts = []
        for i in range(1, len(spot_rates)):
            spot_discounts.append(1 / (1 + spot_rates[i]) ** (dates_of_premium_payments[i]))
        
        for t in range(1, len(dates_of_premium_payments) - 1):
            Premium_Leg_1 = (Fwd_Spread / frequency) * spot_discounts[t] * surv_prob[t]
            
        new_spot_discounts = []
        for i in range(len(new_rates) - 1):
            new_spot_discounts.append(1 / (1 + new_rates[i]) ** (new_dates[i]))
        
        res_1 = 0
        for i in range(1, len(new_spot_discounts) - 1):
            res_1 += (new_spot_discounts[i] + new_spot_discounts[i + 1]) * (new_probabilities[i] - new_probabilities[i + 1])
        Protection_Leg_1 *= res_1
        
        

        
        V = Protection_Leg_1 - Premium_Leg_1
        
        if side == 'buy':
            return V
        elif side == 'sell':
            return -V
        elif side == 'spread':
            return Fwd_Spread
        
        
        

    
#trade_date -- дата начала контракта, spread -- спред в бипсах, term -- срок в годах (от даты начала), 
#frequency -- частота выплаты спреда в год, currency -- валюта оцениваемого CDS 
#(она может отличаться от котируемой на рынке), 
#side -- buy/sell (здесь buy означает покупку защиты = buy protection)
#(!) дата оценки (value date) может быть получена из экземпляра рыночных данных


In [7]:
# проверим работу Американсокго и Европейского CDS

In [8]:
Credit.price_european_cds(md, 'JAGUAR', 4.5, 2, 'EUR', 'sell', 2) 

-0.08424126170750987

In [9]:
Credit.price_american_cds(md, 'JAGUAR', 4.5, 2, 'EUR', 'sell', 2)

-0.05281367168079126

In [10]:
# Проверим вероятности на адекватность
Credit.get_interpolated_sp(md, 'JAGUAR', [0.61, 1, 1.5, 1.7, 2.8, 3, 5])

array([0.98293397, 0.9672161 , 0.94145066, 0.93133771, 0.86392729,
       0.85129207, 0.71952309])

In [11]:
Credit.get_interpolated_spread(md, 'JAGUAR', [4.5])

array([379.])

def price_cln(md: MarketData, trade_date: datetime.datetime, name: str, coupon: float, 
              term: float, frequency: int, currency: str, pay_accrued: bool, recovery: float),
trade_date -- дата начала ноты, coupon -- купон численный,
term -- срок в годах (от даты начала), frequency -- частота выплаты купона в год, 
currency -- валюта CLN (может отличаться от валюты CDS), 
pay_accrued -- определяет платится или нет накопленный купон при дефолте,
recovery -- возврат CLN при дефолте (может отличаться от рыночного)


In [12]:
Credit.price_cln(md, datetime.datetime(2021,10,28), 'JAGUAR', 1, 4.5, 1, 'EUR', 0, md.recovery)

0.10643185731729549

In [13]:
So = Credit.get_interpolated_spread(md, 'JAGUAR', [4.5])
So[0]

379.0

@staticmethod
def price_cds(md: MarketData, trade_date: datetime.datetime, name: str, spread: float, term: float, frequency: int, currency: str, side: str),
trade_date -- дата начала контракта, spread -- спред в бипсах, term -- срок в годах (от даты начала), frequency -- частота выплаты спреда в год, currency -- валюта оцениваемого CDS (она может отличаться от котируемой на рынке), side -- buy/sell (здесь buy означает покупку защиты = buy protection)
(!) дата оценки (value date) может быть получена из экземпляра рыночных данны


In [14]:
Credit.price_cds(md, datetime.datetime(2021,10,28), 'JAGUAR', So[0], 4.5, 2, 'EUR', 'sell') 

0.032588135651612504

Добавил функцию линейной интерполяции спредов в класс Credit. В price_cds нужен спред на 4.5 года, его можно получить из этой функции так: Credit.get_interpolated_spread(md, 'JAGUAR', [4.5])

### 1  Какой максимальный купон можно предложить клиенту, если деск хочет оставить себе 3% маржи?

In [15]:
from sympy.solvers import solve
from sympy import Symbol

In [16]:
x = Symbol('x')
CDS = Credit.price_cds(md, datetime.datetime(2021,10,28), 'JAGUAR', x, 4.5, 2, 'EUR', 'sell') 
CLN = Credit.price_cln(md, datetime.datetime(2021,10,28), 'JAGUAR', x, 4.5, 2, 'EUR', 1, md.recovery)
Coupon1 = solve(CLN - 0.03, x)
Coupon2 = solve(CDS - 0.03, x)
Coupon1[0], Coupon2[0] / 100

(-0.870101794156011, 3.73322392029813)

In [17]:
x = Symbol('x')
CDS = Credit.price_cds(md, datetime.datetime(2021,10,28), 'JAGUAR', x, 4.5, 2, 'USD', 'sell') 
CLN = Credit.price_cln(md, datetime.datetime(2021,10,28), 'JAGUAR', x, 4.5, 2, 'USD', 1, md.recovery)
Coupon1 = solve(CLN - 0.03, x)
Coupon2 = solve(CDS - 0.03, x)
Coupon1[0], Coupon2[0] / 100

(-0.981890881852631, 3.73172404365534)

1) Возможно можно уменьшить частоту выплаты купона в год, например, взять одну выплату в год, либо запросить всю выплату в дату окончания контракта. Тогда есть вероятность изменения процетных ставок и увеличения купона.

2) Так же можно покупать в разных валютах и засчет изменения курса получать прибыль

3) Возможно у эмитентов с более низким кредитным качеством более высокая вероятность дефолта, соответственоо такой CLN должен иметь бОльший купон

Сравним их значения в зависимости от купона, валюты..

In [18]:
Credit.price_european_cds(md, 'JAGUAR', 4.5, 2, 'EUR', 'sell', 2) 

-0.08424126170750987

In [19]:
Credit.price_american_cds(md, 'JAGUAR', 4.5, 2, 'EUR', 'sell', 2)

-0.05281367168079126

In [20]:
Credit.price_european_cds(md, 'JAGUAR', 4.5, 2, 'EUR', 'sell', 1) 

-0.1279238682371432

In [21]:
Credit.price_american_cds(md, 'JAGUAR', 4.5, 2, 'EUR', 'sell', 1)

-0.09649627821042459

In [22]:
Credit.price_european_cds(md, 'JAGUAR', 4.5, 2, 'USD', 'sell', 2) 

-0.07408993503300847

In [23]:
Credit.price_american_cds(md, 'JAGUAR', 4.5, 2, 'USD', 'sell', 2)

-0.05110623946725841

In [24]:
Credit.price_european_cds(md, 'JAGUAR', 4.5, 2, 'USD', 'sell', 1) 

-0.11627708277398877

In [25]:
Credit.price_american_cds(md, 'JAGUAR', 4.5, 2, 'USD', 'sell', 1)

-0.09329338720823871

Как видим - значение Американского CDS немного больше Европейского

In [26]:
# найдем значение купона для первой задачи с прошлой недели и сравним с тем, что получилось тогда

In [27]:
# European, USD
x = Symbol('x')
CDS = Credit.price_european_cds(md, 'JAGUAR', 4.5, 2, 'USD', 'sell', x) 
Coupon = solve(CDS - 0.03, x)
Coupon

[4.46733758044268]

In [28]:
# European, EUR
x = Symbol('x')
CDS = Credit.price_european_cds(md, 'JAGUAR', 4.5, 2, 'EUR', 'sell', x) 
Coupon = solve(CDS - 0.03, x)
Coupon

[4.61525743959468]

In [29]:
# American, USD
x = Symbol('x')
CDS = Credit.price_american_cds(md, 'JAGUAR', 4.5, 2, 'USD', 'sell', x) 
Coupon = solve(CDS - 0.03, x)
Coupon

[3.92253432171411]

In [30]:
# American, EUR
x = Symbol('x')
CDS = Credit.price_american_cds(md, 'JAGUAR', 4.5, 2, 'EUR', 'sell', x) 
Coupon = solve(CDS -  0.03, x)
Coupon

[3.89580426306778]

In [72]:
Credit.price_cmcds(md, 'JAGUAR', 5, 1, 'EUR', 'buy', 0)

0.15569210243641626

In [49]:
Credit.price_cmcds(md, 'JAGUAR', 5, 1, 'EUR', 'sell', 0)

-0.15569210243641626

Поскольку protection leg в cmcds и cds одинаковы - можно купить cmcds и продать cds или наоборот, чтобы избавиться от этой части. Тогда останется разница только между премиальными ногами, которые принимают на себя только риск спреда. 

In [33]:
Credit.price_cmcds(md, 'JAGUAR', 5, 1, 'EUR', 'buy', 1)

0.11774397341692946

In [34]:
Credit.price_cmcds(md, 'JAGUAR', 5, 1, 'EUR', 'buy', 1)

0.11713121556944374

In [35]:
Credit.price_cmcds(md, 'JAGUAR', 5, 1, 'EUR', 'buy', 1)

0.11719121173556873

In [36]:
Credit.price_cmcds(md, 'JAGUAR', 5, 1, 'EUR', 'buy', 1)

0.11727740727451923

In [37]:
Credit.price_cmcds(md, 'JAGUAR', 5, 1, 'EUR', 'buy', 1)

0.11753817018824139

In [38]:
Credit.price_cmcds(md, 'JAGUAR', 5, 1, 'EUR', 'buy', 1)

0.11765796996810485

In [39]:
Credit.price_cmcds(md, 'JAGUAR', 5, 1, 'EUR', 'sell', 1)

-0.11689033963301658

In [40]:
Credit.price_cmcds(md, 'JAGUAR', 5, 1, 'EUR', 'sell', 1)

-0.11700797491610111

In [41]:
Credit.price_cmcds(md, 'JAGUAR', 5, 1, 'EUR', 'sell', 1)

-0.11767438306751796

In [42]:
Credit.price_cmcds(md, 'JAGUAR', 5, 1, 'EUR', 'sell', 1)

-0.11726318394877844

In [43]:
Credit.price_cmcds(md, 'JAGUAR', 5, 1, 'EUR', 'sell', 1)

-0.11718934220656255

In [44]:
Credit.price_cmcds(md, 'JAGUAR', 5, 1, 'EUR', 'sell', 1)

-0.11784497863779698

In [45]:
mean_lognorm_cmcds_buy = 0
mean_lognorm_cmcds_sell = 0
for i in range(100000):
    mean_lognorm_cmcds_buy += Credit.price_cmcds(md, 'JAGUAR', 5, 1, 'EUR', 'buy', 1)
    mean_lognorm_cmcds_sell += Credit.price_cmcds(md, 'JAGUAR', 5, 1, 'EUR', 'sell', 1)
mean_lognorm_cmcds_buy /= 100000
mean_lognorm_cmcds_sell /= 100000
mean_lognorm_cmcds_buy, mean_lognorm_cmcds_sell

(0.11737493638778845, -0.11737579963579728)

In [46]:
diff1 = Credit.price_cmcds(md, 'JAGUAR', 5, 1, 'EUR', 'buy', 0) - mean_lognorm_cmcds_buy
diff2 = Credit.price_cmcds(md, 'JAGUAR', 5, 1, 'EUR', 'sell', 0) - mean_lognorm_cmcds_sell
diff1, diff2

(0.03831716604862781, -0.038316302800618984)